In [2]:
import pandas as pd
import os
import numpy as np
cities = ["ATL_stations","CH_stations","LA_stations","NYC_stations","SD_stations","SF_stations"]
#Aggregate Station Location tuples 
stations_data = pd.DataFrame()
for city in cities:
    path = 'station_locations/%s.txt' % city
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True,header=None,error_bad_lines=False)
        frame['city'] = city
        stations_data = stations_data.append(frame,ignore_index=True)
stations_data = stations_data.rename(columns={0:"distance",1:"station_name"})

In [3]:
ATLshortlist = stations_data[(stations_data.city == "ATL_stations") & (stations_data.distance <= 50)][['station_name']]
CHshortlist = stations_data[(stations_data.city == "CH_stations") & (stations_data.distance <= 50)][['station_name']]
LAshortlist = stations_data[(stations_data.city == "LA_stations") & (stations_data.distance <= 50)][['station_name']]
NYCshortlist = stations_data[(stations_data.city == "NYC_stations") & (stations_data.distance <= 50)][['station_name']]
SDshortlist = stations_data[(stations_data.city == "SD_stations") & (stations_data.distance <= 50)][['station_name']]
SFshortlist = stations_data[(stations_data.city == "SF_stations") & (stations_data.distance <= 50)][['station_name']]